# **Machine Learning for Supply Chains**

## **Course 1: Fundamentals of Machine Learning for Supply Chains**

### **What is PuLP**

PuLP is a Python package that specializes in discrete optimization. For example, suppose we want to maximize $x \cdot y$ given that $2x + y = 3$. 

If we allow any number as a solution, then calculus tells us that we would get $x = \frac{3}{4}$ and $y = \frac{3}{2}$. 

However, suppose we are only allowed to have integers for solutions (i.e., no fractions). One might expect the answer to be integers close to these fractions (e.g., $x = 1$ and $y = 1$), but it's not always this simple. We would use PuLP in place of calculus for problems like this.

#### **Case Study: Simple Scheduling Application using Linear Programming (PuLP)**

With reference to [Medium article about PuLP](#Optimization-with-PuLP-in-Python---Getting-Started), I have applied the code mentioned in the article as follows:

##### **Problem Overview**

We run a 24-hour lemonade stand with two products:

- **Iced Lemonade**

- **Frozen Lemonade Slushies**

Each product has a **processing time** and a **forecasted hourly demand**.

Our goal is to find out:

- How many staff members are needed for each hour to **meet customer demand**.

- How to **minimize staffing costs**, while ensuring all customer orders can be fulfilled.

##### **Step 1: Import Libraries and Define Inputs**

In [5]:
# importing the necessary libraries
import pandas as pd 
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus


In [6]:
# Define the 24 hours in a day
hours = range(24)

# Define expected hourly demand for each product
demand_iced = pd.Series(
    [7, 11, 8, 8, 5, 3, 8, 20, 52, 56, 85, 76, 102, 67, 82, 68, 65, 56, 50, 43, 47, 23, 29, 18]
)

demand_slushy = pd.Series(
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 38, 84, 93, 82, 93, 75, 70, 62, 22, 27, 17, 22, 0, 0, 0]
)

# Processing time in hours per product
processing_time_iced = 2 / 60   # 2 minutes per iced lemonade
processing_time_slushy = 5 / 60 # 5 minutes per slushy


##### **Step 2: Define the Optimization Problem**

We are going to use **PuLP** to minimize the total staff cost, assuming:

- Each staff member costs $15 per hour.

- We need to satisfy customer demand using available staff hours.

In [7]:
# Define the LP problem
prob = LpProblem("Simple_Scheduling_Application", LpMinimize)

# Define decision variables for number of staff needed each hour
staff_needed = LpVariable.dicts("staff_hour", hours, lowBound=0, cat='Continuous')


##### **Step 3: Set Objective Function**

Our objective is to **minimize the total hourly cost** of staff.

In [8]:
# Objective: Minimize total staffing cost
prob += lpSum([15 * staff_needed[i] for i in hours]), "Total_Cost"

##### **Step 4: Add Constraints**

We add two main constraints for each hour:

1. At least one person must be present at all times.

2. The total staffing must be enough to handle demand.

In [9]:
# Add constraints for each hour
for hour in hours:
    # Minimum 1 staff member at all hours
    prob += staff_needed[hour] >= 1, f"MinStaff_{hour}"
    
    # Staff must be enough to meet demand
    total_demand = processing_time_iced * demand_iced[hour] + processing_time_slushy * demand_slushy[hour]
    prob += staff_needed[hour] >= total_demand, f"DemandConstraint_{hour}"


##### **Step 5: Solve the Problem**

In [10]:
# Solve the LP problem
status = prob.solve()

# Print the solution status
print("Status:", LpStatus[status])

# Print staff needed for each hour
for v in prob.variables():
    print(v.name, "=", round(v.varValue, 2))


Status: Optimal
staff_hour_0 = 1.0
staff_hour_1 = 1.0
staff_hour_10 = 9.83
staff_hour_11 = 10.28
staff_hour_12 = 10.23
staff_hour_13 = 9.98
staff_hour_14 = 8.98
staff_hour_15 = 8.1
staff_hour_16 = 7.33
staff_hour_17 = 3.7
staff_hour_18 = 3.92
staff_hour_19 = 2.85
staff_hour_2 = 1.0
staff_hour_20 = 3.4
staff_hour_21 = 1.0
staff_hour_22 = 1.0
staff_hour_23 = 1.0
staff_hour_3 = 1.0
staff_hour_4 = 1.0
staff_hour_5 = 1.0
staff_hour_6 = 1.0
staff_hour_7 = 1.0
staff_hour_8 = 1.73
staff_hour_9 = 5.03


##### **Improved Version with Staffing Constraints**

##### **Problem Update**

We now introduce **limited staff availability**:

- We have only **5 employees**.

- Each can work **up to 8 hours**, totaling 40 hours/day.

We also account for **lost sales** when demand is not met.

##### Define New Decision Variables

In [11]:
# Re-initialize the problem with new constraints
prob = LpProblem("Scheduling_With_Staff_Limit", LpMinimize)

# Define integer decision variables for staff and lost sales
staff_needed = LpVariable.dicts("staff_hour", hours, lowBound=0, cat='Integer')
lost_iced = LpVariable.dicts("lost_iced", hours, lowBound=0, cat='Integer')
lost_slushy = LpVariable.dicts("lost_slushy", hours, lowBound=0, cat='Integer')

# Lost sales penalty cost
cost_iced = 3
cost_slushy = 5


##### Update the Objective Function

Now, we minimize both:

- The cost of staff, and

- The cost of lost sales.

In [12]:
# Updated objective: Minimize staff + lost sales cost
prob += lpSum([
    15 * staff_needed[i] + cost_iced * lost_iced[i] + cost_slushy * lost_slushy[i]
    for i in hours
]), "Total_Cost_with_Lost_Sales"


##### Update Constraints

In [13]:
# Total working hours for all staff should not exceed 40 (5 employees * 8 hours)
prob += lpSum([staff_needed[i] for i in hours]) <= 5 * 8, "Total_Work_Hours"

for i in hours:
    # Minimum 1 staff per hour
    prob += staff_needed[i] >= 1, f"MinStaff_{i}"
    
    # Max 5 staff at any hour
    prob += staff_needed[i] <= 5, f"MaxStaff_{i}"
    
    # Demand constraint with allowance for lost sales
    adjusted_demand = processing_time_iced * (demand_iced[i] - lost_iced[i]) + \
                      processing_time_slushy * (demand_slushy[i] - lost_slushy[i])
    
    prob += staff_needed[i] >= adjusted_demand, f"DemandConstraint_{i}"


#####  Solve the New Problem

In [14]:
# Solve the LP problem
status = prob.solve()

# Print status
print("Status:", LpStatus[status])

# Print results
for v in prob.variables():
    print(v.name, "=", round(v.varValue, 2))


Status: Optimal
lost_iced_0 = 0.0
lost_iced_1 = 0.0
lost_iced_10 = 0.0
lost_iced_11 = 1.0
lost_iced_12 = 0.0
lost_iced_13 = 0.0
lost_iced_14 = 0.0
lost_iced_15 = 1.0
lost_iced_16 = 0.0
lost_iced_17 = 1.0
lost_iced_18 = 0.0
lost_iced_19 = 1.0
lost_iced_2 = 0.0
lost_iced_20 = 0.0
lost_iced_21 = 0.0
lost_iced_22 = 0.0
lost_iced_23 = 0.0
lost_iced_3 = 0.0
lost_iced_4 = 0.0
lost_iced_5 = 0.0
lost_iced_6 = 0.0
lost_iced_7 = 0.0
lost_iced_8 = 0.0
lost_iced_9 = 1.0
lost_slushy_0 = 0.0
lost_slushy_1 = 0.0
lost_slushy_10 = 106.0
lost_slushy_11 = 111.0
lost_slushy_12 = 111.0
lost_slushy_13 = 60.0
lost_slushy_14 = 48.0
lost_slushy_15 = 85.0
lost_slushy_16 = 76.0
lost_slushy_17 = 8.0
lost_slushy_18 = 11.0
lost_slushy_19 = 22.0
lost_slushy_2 = 0.0
lost_slushy_20 = 5.0
lost_slushy_21 = 0.0
lost_slushy_22 = 0.0
lost_slushy_23 = 0.0
lost_slushy_3 = 0.0
lost_slushy_4 = 0.0
lost_slushy_5 = 0.0
lost_slushy_6 = 0.0
lost_slushy_7 = 0.0
lost_slushy_8 = 9.0
lost_slushy_9 = 24.0
staff_hour_0 = 1.0
staff_hour_1

#### Module 1 - Video: Linear Programming with PuLP I and II

# References

##### [Optimization with PuLP in Python - Getting Started](https://medium.com/@goldkamp.j16/optimization-with-pulp-in-python-getting-started-f6c5b678bf15)
